# 游戏玩家付费金额预测大赛 - 特征工程

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [44]:
path_raw = '../data/raw/'
path_new = '../data/new/'
time_col = 'register_time'

In [57]:
train_part1 = pd.read_csv(path_new + 'train=0.1.csv')
train_part1.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,2320186,2018-02-07 06:51:15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3.833333,0.0,0,0.0
1,2959017,2018-02-10 16:18:10,129429.0,78242.0,0.0,0.0,6000.0,0.0,152480.0,48626.0,...,0,0,0,2,2,2,3.166667,0.0,0,0.0
2,242613,2018-02-09 02:40:08,84839.0,52467.0,0.0,0.0,0.0,0.0,22144.0,29600.0,...,0,0,0,0,0,0,2.666667,0.0,0,0.0
3,1416097,2018-01-26 09:30:30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
4,680469,2018-02-27 10:04:04,28625.0,7200.0,0.0,0.0,0.0,0.0,33625.0,3930.0,...,0,0,0,0,0,0,1.000000,0.0,0,0.0


## float64 四舍五入 转换为 int64——这个不要
可能效果不好，比如在线时间，0.123 不能成为 0  

In [53]:
#对数值型特征，观察其取值范围及直方图
float64_features = train_part1.select_dtypes(include = ["float64"]).columns
for col in float64_features:
    print('\n%s 属性的不同取值和出现的次数' % col)
    print(train_part1[col].value_counts())


wood_add_value 属性的不同取值和出现的次数
0.0           63386
10000.0       23316
11500.0        3612
20125.0        3349
38538.0        1799
43413.0        1644
128252.0       1462
26375.0        1409
84839.0        1266
15500.0        1265
54253.0        1089
18125.0         935
96255.0         902
97666.0         875
129668.0        693
27625.0         651
118252.0        596
211500.0        528
139668.0        498
91817.0         485
210000.0        481
131852.0        443
58404.0         430
21500.0         363
50000.0         348
39038.0         341
28975.0         304
30975.0         299
121852.0        297
220125.0        297
              ...  
474269.0          1
343199.0          1
118568.0          1
42901.0           1
474264.0          1
15177077.0        1
237142.0          1
53189.0           1
3794302.0         1
118572.0          1
686436.0          1
686386.0          1
15176446.0        1
474228.0          1
118563.0          1
2745257.0         1
15175598.0        1
948483.0  

In [54]:
float64_features

Index(['wood_add_value', 'wood_reduce_value', 'stone_add_value',
       'stone_reduce_value', 'ivory_add_value', 'ivory_reduce_value',
       'meat_add_value', 'meat_reduce_value', 'magic_add_value',
       'magic_reduce_value', 'avg_online_minutes', 'pay_price',
       'prediction_pay_price'],
      dtype='object')

In [55]:
for col in float64_features:
    train_part1[col] = train_part1[col].apply(np.round)
    train_part1[col] = train_part1[col].astype(np.int64)
    print('\n%s 属性的不同取值和出现的次数' % col)
    print(train_part1[col].value_counts())


wood_add_value 属性的不同取值和出现的次数
0          63386
10000      23316
11500       3612
20125       3349
38538       1799
43413       1644
128252      1462
26375       1409
84839       1266
15500       1265
54253       1089
18125        935
96255        902
97666        875
129668       693
27625        651
118252       596
211500       528
139668       498
91817        485
210000       481
131852       443
58404        430
21500        363
50000        348
39038        341
28975        304
30975        299
121852       297
220125       297
           ...  
445451         1
193548         1
189454         1
412699         1
87032          1
21838          1
248875         1
523309         1
201776         1
197682         1
220217         1
89081          1
76791          1
256943         1
11223          1
54622          1
703263         1
215995         1
2061244        1
2945996        1
326605         1
809945         1
66546          1
5012445        1
1338335        1
105441         1
1


meat_reduce_value 属性的不同取值和出现的次数
0           81027
2000         9968
6220         2754
400          2658
900          2566
9444         2060
39044        1890
2250         1845
2500         1276
29600        1268
12684        1179
66054        1172
75498        1024
3240         1008
3980          789
6060          587
3220          585
19548         556
19100         515
6470          400
2675          396
7145          382
9656          352
3000          330
5220          291
15066         277
63125         245
7570          235
7220          223
10000         221
            ...  
956930          1
94719           1
88570           1
344569          1
80374           1
1376753         1
125420          1
119275          1
349253          1
891362          1
29151           1
2779823         1
20955           1
801238          1
63950           1
53707           1
47558           1
41413           1
43460           1
33217           1
35264           1
14889399        1
473526       

## register_time 转换为注册天数。

In [41]:
def time_to_days(df, time_col):
    """ 玩家注册时间转换为天数 = 从注册 到抓取数据时的天数。 """
    # 报名开始：6月19日11:00   猜测抓取数据的日期 
    ty = pd.to_datetime('2018-06-11 16:51:15')
    # 采用匿名函数 lambda
    df[time_col] = df[time_col].apply(lambda x: (ty - pd.to_datetime(x)).days)
    df[time_col] = df[time_col].astype(np.int64)

In [58]:
time_to_days(train_part1, time_col)
print('\n%s 属性的不同取值和出现的次数' % col)
print(train_part1[col].value_counts())


prediction_pay_price 属性的不同取值和出现的次数
0.00       224161
0.99         1579
1.98          447
5.98          161
6.97          110
1.99           88
4.99           83
2.97           81
2.98           79
10.98          74
3.97           59
11.97          57
16.96          53
9.99           37
10.97          37
15.97          28
7.97           25
8.96           23
11.96          22
20.96          17
26.95          17
21.95          17
3.96           17
12.96          15
4.96           14
36.95          14
19.99          14
18.95          13
7.96           13
5.96           13
            ...  
110.80          1
490.70          1
405.40          1
265.79          1
181.90          1
106.87          1
167.82          1
298.70          1
126.92          1
169.80          1
326.87          1
22.90           1
1017.64         1
616.78          1
81.92           1
413.64          1
20.89           1
989.55          1
118.74          1
235.74          1
231.88          1
26.93           1
556.65    

In [39]:
test_df = pd.read_csv(path_raw + 'tap_fun_test.csv')
test_df.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14933,2018-03-08 20:27:57,166415.0,138362.0,0,0,0,0,258522,90142,...,0,0,0,0,1,1,1,8.000000,0.0,0
1,14934,2018-03-08 20:29:42,10000.0,600.0,0,0,0,0,10000,400,...,0,0,0,0,0,0,0,0.166667,0.0,0
2,14935,2018-03-08 20:30:13,10000.0,3700.0,0,0,0,0,10000,2000,...,0,0,0,0,0,0,0,17.000000,0.0,0
3,14936,2018-03-08 20:31:41,210000.0,0.0,200000,0,200000,0,610000,0,...,0,0,0,0,0,0,0,1.666667,0.0,0
4,14937,2018-03-08 20:32:07,11500.0,3700.0,0,0,0,0,11000,2000,...,0,0,0,0,0,0,0,0.333333,0.0,0


In [43]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828934 entries, 0 to 828933
Columns: 108 entries, user_id to pay_count
dtypes: float64(4), int64(104)
memory usage: 683.0 MB


In [42]:
time_to_days(test_df, time_col)
print('\n%s 属性的不同取值和出现的次数' % time_col)
print(test_df[time_col].value_counts())


register_time 属性的不同取值和出现的次数
93    119941
92     69635
90     60598
86     53363
87     47669
89     47323
85     45907
91     45539
82     43898
95     43345
81     43314
84     42111
88     41895
83     41810
94     41230
96     24947
80     16409
Name: register_time, dtype: int64


In [51]:
print('train =', train_part1[time_col].mean())
print('test =', test_df[time_col].mean())
# 训练集的玩家注册时间长，测试集的时间短，预测值按比例看看。

train = 117.5991083916084
test = 88.68314727107345


In [73]:
train_part1.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,2320186,124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3.833333,0.0,0,0.0
1,2959017,121,129429.0,78242.0,0.0,0.0,6000.0,0.0,152480.0,48626.0,...,0,0,0,2,2,2,3.166667,0.0,0,0.0
2,242613,122,84839.0,52467.0,0.0,0.0,0.0,0.0,22144.0,29600.0,...,0,0,0,0,0,0,2.666667,0.0,0,0.0
3,1416097,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
4,680469,104,28625.0,7200.0,0.0,0.0,0.0,0.0,33625.0,3930.0,...,0,0,0,0,0,0,1.000000,0.0,0,0.0


In [74]:
# train=0.1.csv 特征工程后的，注册日期转换为天数
train_part1.to_csv(path_new + 'train=0.1-fe.csv', index=False)
train_part1.shape

(228800, 109)

In [77]:
test_df.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14933,94,166415.0,138362.0,0,0,0,0,258522,90142,...,0,0,0,0,1,1,1,8.000000,0.0,0
1,14934,94,10000.0,600.0,0,0,0,0,10000,400,...,0,0,0,0,0,0,0,0.166667,0.0,0
2,14935,94,10000.0,3700.0,0,0,0,0,10000,2000,...,0,0,0,0,0,0,0,17.000000,0.0,0
3,14936,94,210000.0,0.0,200000,0,200000,0,610000,0,...,0,0,0,0,0,0,0,1.666667,0.0,0
4,14937,94,11500.0,3700.0,0,0,0,0,11000,2000,...,0,0,0,0,0,0,0,0.333333,0.0,0


In [78]:
# tap_fun_test.csv 特征工程后的，注册日期转换为天数
test_df.to_csv(path_new + 'tap_fun_test-fe.csv', index=False)
test_df.shape

(828934, 108)

## 特征之间的相关性

In [59]:
train_part1.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,2320186,124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3.833333,0.0,0,0.0
1,2959017,121,129429.0,78242.0,0.0,0.0,6000.0,0.0,152480.0,48626.0,...,0,0,0,2,2,2,3.166667,0.0,0,0.0
2,242613,122,84839.0,52467.0,0.0,0.0,0.0,0.0,22144.0,29600.0,...,0,0,0,0,0,0,2.666667,0.0,0,0.0
3,1416097,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
4,680469,104,28625.0,7200.0,0.0,0.0,0.0,0.0,33625.0,3930.0,...,0,0,0,0,0,0,1.000000,0.0,0,0.0


In [63]:
%%time
# 不计算 id
train = train_part1.drop(['user_id'], axis=1)
#get the names of all the columns
cols=train.columns 

# Calculates pearson co-efficient for all combinations，通常认为相关系数 > 0.5 的为强相关
data_corr = train.corr().abs()

plt.subplots(figsize=(80, 30))
sns.heatmap(data_corr,annot=True)

# Mask unimportant features
sns.heatmap(data_corr, mask=data_corr < 1, cbar=False)

plt.savefig(path_new + 'game_coor_2.png' )
plt.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Wall time: 48.7 s


In [64]:
data_corr

,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
register_time,1.000000,0.015468,0.017886,0.003306,0.002515,0.002601,0.000737,0.018287,0.017993,0.005436,...,0.021651,0.017432,0.013266,0.052453,0.052417,0.049829,0.000705,0.002335,0.005901,0.001568
wood_add_value,0.015468,1.000000,0.955549,0.975761,0.959829,0.971689,0.939044,0.899524,0.930127,0.417139,...,0.261855,0.254299,0.283086,0.340144,0.339379,0.336601,0.359576,0.891059,0.684707,0.732175
wood_reduce_value,0.017886,0.955549,1.000000,0.896274,0.947048,0.901195,0.902088,0.836164,0.933394,0.446593,...,0.317256,0.311041,0.344312,0.422640,0.421671,0.418304,0.443071,0.870434,0.725854,0.712524
stone_add_value,0.003306,0.975761,0.896274,1.000000,0.970473,0.990678,0.960540,0.897857,0.918941,0.383185,...,0.182839,0.182185,0.206033,0.207292,0.206704,0.205140,0.237000,0.905230,0.634133,0.737585
stone_reduce_value,0.002515,0.959829,0.947048,0.970473,1.000000,0.968490,0.973558,0.868531,0.950021,0.412654,...,0.209252,0.212121,0.239419,0.240672,0.239958,0.238298,0.275972,0.924619,0.672541,0.747822
ivory_add_value,0.002601,0.971689,0.901195,0.990678,0.968490,1.000000,0.978984,0.887001,0.909886,0.387193,...,0.182288,0.179436,0.203775,0.201083,0.200144,0.198907,0.230688,0.909818,0.643655,0.746421
ivory_reduce_value,0.000737,0.939044,0.902088,0.960540,0.973558,0.978984,1.000000,0.838807,0.887431,0.380642,...,0.147502,0.144557,0.167790,0.142057,0.141165,0.140297,0.177385,0.911758,0.603684,0.743139
meat_add_value,0.018287,0.899524,0.836164,0.897857,0.868531,0.887001,0.838807,1.000000,0.878952,0.367632,...,0.252310,0.246176,0.272127,0.326114,0.325393,0.322319,0.343299,0.792775,0.623492,0.650798
meat_reduce_value,0.017993,0.930127,0.933394,0.918941,0.950021,0.909886,0.887431,0.878952,1.000000,0.424926,...,0.325851,0.322120,0.353914,0.418786,0.417823,0.413869,0.439080,0.858870,0.713156,0.700602
magic_add_value,0.005436,0.417139,0.446593,0.383185,0.412654,0.387193,0.380642,0.367632,0.424926,1.000000,...,0.222911,0.176424,0.192903,0.237562,0.236143,0.235897,0.262496,0.401094,0.349691,0.328770


In [65]:
# 根据 prediction_pay_price 相关性排序，升序
sort_coor = data_corr.sort_values('prediction_pay_price')
sort_coor

,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
register_time,1.000000,0.015468,0.017886,0.003306,0.002515,0.002601,0.000737,0.018287,0.017993,0.005436,...,0.021651,0.017432,0.013266,0.052453,0.052417,0.049829,0.000705,0.002335,0.005901,0.001568
sr_scout_level,0.000056,0.189367,0.236684,0.101103,0.115647,0.098680,0.063674,0.183111,0.235687,0.142357,...,0.338842,0.280422,0.300109,0.470115,0.469583,0.460462,0.444456,0.071852,0.223884,0.073557
sr_outpost_tier_2_level,0.057116,0.232405,0.291011,0.122906,0.138326,0.117131,0.068516,0.234958,0.295558,0.163646,...,0.382797,0.330830,0.347601,0.608595,0.607836,0.595544,0.552843,0.078585,0.252271,0.079228
bd_magic_coin_tree_level,0.046698,0.230966,0.283127,0.121308,0.134433,0.112989,0.075395,0.228544,0.284352,0.164629,...,0.308300,0.257934,0.276990,0.512746,0.512272,0.501715,0.494317,0.082360,0.222832,0.080164
bd_market_level,0.025641,0.241943,0.303822,0.128008,0.145494,0.119072,0.075927,0.240209,0.307429,0.179059,...,0.332269,0.284699,0.308606,0.547711,0.547231,0.537790,0.538260,0.088668,0.270815,0.089973
bd_training_hut_level,0.088653,0.233329,0.286916,0.125587,0.140642,0.117817,0.080221,0.230419,0.289846,0.165255,...,0.321457,0.264616,0.283770,0.518120,0.517777,0.506903,0.507974,0.096813,0.252431,0.093576
bd_warehouse_level,0.103180,0.258533,0.316276,0.139118,0.153493,0.129641,0.087639,0.255302,0.315861,0.167614,...,0.346191,0.295959,0.316902,0.580084,0.579561,0.567062,0.558613,0.097672,0.266519,0.096393
bd_healing_lodge_level,0.102239,0.236204,0.292203,0.125613,0.142377,0.119365,0.081325,0.233026,0.292708,0.156365,...,0.331950,0.281315,0.300240,0.555109,0.554739,0.542445,0.530526,0.102422,0.261770,0.096621
sr_rss_d_prod_level,0.030518,0.212916,0.265570,0.126602,0.145391,0.121498,0.079919,0.208194,0.260010,0.143170,...,0.274834,0.263025,0.283786,0.527129,0.527080,0.525254,0.475150,0.105847,0.293004,0.096831
sr_troop_consumption_level,0.022756,0.211065,0.260378,0.129663,0.147731,0.125378,0.082907,0.205293,0.252583,0.138481,...,0.255780,0.252170,0.274438,0.485800,0.486207,0.483800,0.448150,0.108849,0.309640,0.100955


In [70]:
# 取相关性 > 0.3 
threashhold = 0.3
high_coor = sort_coor.loc[sort_coor['prediction_pay_price'] > threashhold, ['pay_price','pay_count','prediction_pay_price']]
high_coor

,pay_price,pay_count,prediction_pay_price
sr_infantry_atk_level,0.350232,0.656094,0.320464
magic_add_value,0.401094,0.349691,0.328770
cavalry_add_value,0.390930,0.538951,0.333436
sr_rss_e_gather_level,0.412443,0.431104,0.335987
sr_cavalry_atk_level,0.372536,0.667096,0.341669
sr_march_size_level,0.489823,0.204157,0.344306
sr_outpost_tier_4_level,0.489823,0.204157,0.344306
sr_guest_troop_capacity_level,0.489823,0.204157,0.344306
sr_shaman_atk_level,0.387948,0.657870,0.354319
sr_outpost_tier_3_level,0.389628,0.530950,0.390737


In [71]:
# 只有 48 个特征，与目标值相关性 >  0.3
high_coor.shape

(48, 3)

In [105]:
# 相关性 >  0.3 的特征名称 
high_cols = high_coor.axes[0]
high_cols

Index(['sr_infantry_atk_level', 'magic_add_value', 'cavalry_add_value',
       'sr_rss_e_gather_level', 'sr_cavalry_atk_level', 'sr_march_size_level',
       'sr_outpost_tier_4_level', 'sr_guest_troop_capacity_level',
       'sr_shaman_atk_level', 'sr_outpost_tier_3_level',
       'sr_troop_attack_level', 'shaman_add_value', 'infantry_add_value',
       'sr_gathering_march_speed_level', 'sr_pvp_march_speed_level',
       'sr_shaman_tier_4_level', 'sr_alliance_march_speed_level',
       'sr_cavalry_tier_4_level', 'sr_cavalry_tier_3_level',
       'sr_infantry_tier_3_level', 'sr_shaman_tier_3_level',
       'sr_infantry_tier_4_level', 'building_acceleration_add_value',
       'pay_count', 'sr_troop_defense_level', 'sr_cavalry_hp_level',
       'building_acceleration_reduce_value', 'sr_shaman_def_level',
       'sr_infantry_def_level', 'sr_shaman_hp_level', 'sr_cavalry_def_level',
       'meat_add_value', 'reaserch_acceleration_add_value',
       'sr_infantry_hp_level', 'training_accelera

In [109]:
high_cols = high_cols.insert(0, 'user_id')
high_cols

Index(['user_id', 'sr_infantry_atk_level', 'magic_add_value',
       'cavalry_add_value', 'sr_rss_e_gather_level', 'sr_cavalry_atk_level',
       'sr_march_size_level', 'sr_outpost_tier_4_level',
       'sr_guest_troop_capacity_level', 'sr_shaman_atk_level',
       'sr_outpost_tier_3_level', 'sr_troop_attack_level', 'shaman_add_value',
       'infantry_add_value', 'sr_gathering_march_speed_level',
       'sr_pvp_march_speed_level', 'sr_shaman_tier_4_level',
       'sr_alliance_march_speed_level', 'sr_cavalry_tier_4_level',
       'sr_cavalry_tier_3_level', 'sr_infantry_tier_3_level',
       'sr_shaman_tier_3_level', 'sr_infantry_tier_4_level',
       'building_acceleration_add_value', 'pay_count',
       'sr_troop_defense_level', 'sr_cavalry_hp_level',
       'building_acceleration_reduce_value', 'sr_shaman_def_level',
       'sr_infantry_def_level', 'sr_shaman_hp_level', 'sr_cavalry_def_level',
       'meat_add_value', 'reaserch_acceleration_add_value',
       'sr_infantry_hp_level', 

In [114]:
def get_high_coor(df, cols):
    ''' 获取指定的特征，返回 dataframe 副本 '''
    df_high_coor = df[high_cols]
    print('shape =', df_high_coor.shape)
    return df_high_coor

# 训练集和测试集都要
train_high_coor = get_high_coor(train_part1, high_cols)
train_high_coor.head()

shape = (228800, 49)


,user_id,sr_infantry_atk_level,magic_add_value,cavalry_add_value,sr_rss_e_gather_level,sr_cavalry_atk_level,sr_march_size_level,sr_outpost_tier_4_level,sr_guest_troop_capacity_level,sr_shaman_atk_level,...,wood_reduce_value,general_acceleration_add_value,wood_add_value,stone_add_value,general_acceleration_reduce_value,ivory_reduce_value,ivory_add_value,stone_reduce_value,pay_price,prediction_pay_price
0,2320186,0,0.0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,2959017,0,0.0,0,0,0,0,0,0,0,...,78242.0,0,129429.0,0.0,0,0.0,6000.0,0.0,0.0,0.0
2,242613,0,0.0,0,0,0,0,0,0,0,...,52467.0,60,84839.0,0.0,1,0.0,0.0,0.0,0.0,0.0
3,1416097,0,0.0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,680469,0,0.0,0,0,0,0,0,0,0,...,7200.0,0,28625.0,0.0,0,0.0,0.0,0.0,0.0,0.0


In [116]:
# 测试集没有 prediction_pay_price 需要去掉 
loc = high_cols.get_loc('prediction_pay_price')
high_cols = high_cols.delete(loc)
test_high_coor = get_high_coor(test_df, high_cols)
test_high_coor.head()

shape = (828934, 48)


,user_id,sr_infantry_atk_level,magic_add_value,cavalry_add_value,sr_rss_e_gather_level,sr_cavalry_atk_level,sr_march_size_level,sr_outpost_tier_4_level,sr_guest_troop_capacity_level,sr_shaman_atk_level,...,training_acceleration_reduce_value,wood_reduce_value,general_acceleration_add_value,wood_add_value,stone_add_value,general_acceleration_reduce_value,ivory_reduce_value,ivory_add_value,stone_reduce_value,pay_price
0,14933,0,0,0,0,0,0,0,0,0,...,0,138362.0,3,166415.0,0,0,0,0,0,0.0
1,14934,0,0,0,0,0,0,0,0,0,...,0,600.0,0,10000.0,0,0,0,0,0,0.0
2,14935,0,0,0,0,0,0,0,0,0,...,0,3700.0,0,10000.0,0,0,0,0,0,0.0
3,14936,0,60000,0,0,0,0,0,0,0,...,0,0.0,660,210000.0,200000,0,0,200000,0,0.0
4,14937,0,0,0,0,0,0,0,0,0,...,0,3700.0,0,11500.0,0,0,0,0,0,0.0


In [117]:
# train=0.1-fe.csv 取相关性 > 0.3的特征47个
train_high_coor.to_csv(path_new + 'train=0.1-fe-coor0.3.csv', index=False)
train_high_coor.shape

(228800, 49)

In [118]:
# tap_fun_test-fe.csv 取相关性 > 0.3的特征47个
test_high_coor.to_csv(path_new + 'tap_fun_test-fe-coor0.3.csv', index=False)
test_high_coor.shape

(828934, 48)

In [67]:
col = 'sr_rss_help_bonus_level'
print('\n%s 属性的不同取值和出现的次数' % col)
print(train[col].value_counts())


sr_rss_help_bonus_level 属性的不同取值和出现的次数
0    228800
Name: sr_rss_help_bonus_level, dtype: int64


In [68]:
col = 'sr_rss_help_bonus_level'
print('\n%s 属性的不同取值和出现的次数' % col)
print(test_df[col].value_counts())


sr_rss_help_bonus_level 属性的不同取值和出现的次数
0    828931
1         3
Name: sr_rss_help_bonus_level, dtype: int64


In [69]:
# sr_rss_help_bonus_level > 0 的是否付费？
tmp_df = test_df.loc[test_df['sr_rss_help_bonus_level'] > 0, ['sr_rss_help_bonus_level','pay_price','pay_count','prediction_pay_price']]
tmp_df.head()

,sr_rss_help_bonus_level,pay_price,pay_count,prediction_pay_price
364418,1,4223.30,70,NaN
393756,1,2903.49,51,NaN
623765,1,14495.22,178,NaN


## 添加重要特征（top 10）的多项式

In [75]:
sort2_coor = data_corr.sort_values('prediction_pay_price', ascending = False)
sort2_coor

,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
prediction_pay_price,0.001568,0.732175,0.712524,0.737585,0.747822,0.746421,0.743139,0.650798,0.700602,0.328770,...,0.126395,0.129522,0.147405,0.140424,0.140058,0.137470,0.189115,0.803923,0.598916,1.000000
pay_price,0.002335,0.891059,0.870434,0.905230,0.924619,0.909818,0.911758,0.792775,0.858870,0.401094,...,0.145753,0.143979,0.165755,0.146001,0.145523,0.143840,0.192773,1.000000,0.703309,0.803923
stone_reduce_value,0.002515,0.959829,0.947048,0.970473,1.000000,0.968490,0.973558,0.868531,0.950021,0.412654,...,0.209252,0.212121,0.239419,0.240672,0.239958,0.238298,0.275972,0.924619,0.672541,0.747822
ivory_add_value,0.002601,0.971689,0.901195,0.990678,0.968490,1.000000,0.978984,0.887001,0.909886,0.387193,...,0.182288,0.179436,0.203775,0.201083,0.200144,0.198907,0.230688,0.909818,0.643655,0.746421
ivory_reduce_value,0.000737,0.939044,0.902088,0.960540,0.973558,0.978984,1.000000,0.838807,0.887431,0.380642,...,0.147502,0.144557,0.167790,0.142057,0.141165,0.140297,0.177385,0.911758,0.603684,0.743139
general_acceleration_reduce_value,0.007261,0.900184,0.943734,0.860485,0.917314,0.867228,0.871459,0.792501,0.901077,0.437235,...,0.285632,0.289574,0.320716,0.352074,0.351468,0.347311,0.401509,0.895368,0.762830,0.740558
stone_add_value,0.003306,0.975761,0.896274,1.000000,0.970473,0.990678,0.960540,0.897857,0.918941,0.383185,...,0.182839,0.182185,0.206033,0.207292,0.206704,0.205140,0.237000,0.905230,0.634133,0.737585
wood_add_value,0.015468,1.000000,0.955549,0.975761,0.959829,0.971689,0.939044,0.899524,0.930127,0.417139,...,0.261855,0.254299,0.283086,0.340144,0.339379,0.336601,0.359576,0.891059,0.684707,0.732175
general_acceleration_add_value,0.010754,0.887944,0.932864,0.841759,0.895895,0.846883,0.838047,0.788271,0.899093,0.443796,...,0.305188,0.303669,0.336389,0.407277,0.406502,0.402701,0.445902,0.886887,0.793226,0.725020
wood_reduce_value,0.017886,0.955549,1.000000,0.896274,0.947048,0.901195,0.902088,0.836164,0.933394,0.446593,...,0.317256,0.311041,0.344312,0.422640,0.421671,0.418304,0.443071,0.870434,0.725854,0.712524


In [92]:
def get_top_col(df, col, top_num):
    """ 获取 df 中指定特征 col 的相关性最高的 top 个特征 """
    # 相关性阈值 the first one is itself,from 1-top
    threshold = df[col].iloc[top_num]  
    print('threshold=', threshold)

    # 与目标值相关性最高的 10 个特征+自身 = 11 个
    top_cols = (df[col][df[col] > threshold]).axes
    print('top_cols=', top_cols)
    return top_cols

# 与目标值相关性最高的 10 个特征+自身 = 11 个
top_num = 11
top_cols = get_top_col(sort2_coor, 'prediction_pay_price', top_num)

threshold= 0.700602111296
top_cols= [Index(['prediction_pay_price', 'pay_price', 'stone_reduce_value',
       'ivory_add_value', 'ivory_reduce_value',
       'general_acceleration_reduce_value', 'stone_add_value',
       'wood_add_value', 'general_acceleration_add_value', 'wood_reduce_value',
       'training_acceleration_reduce_value'],
      dtype='object')]


In [94]:
# Create new features
# 3* Polynomials on the top 10 existing features
# 对相关性最高的 10 个特征，计算多项式
def Polynomials_top10(df, top_cols, top_num):
    df_top10 = df.copy()
#     因为第一个是自身，所以从索引 1 第二个开始处理 
    for i in range(1,top_num):
#         新的特征名称
        new_cols_2 = top_cols[0][i] + '_s' + str(2)
        new_cols_3 = top_cols[0][i] + '_s' + str(3)
        new_cols_sq = top_cols[0][i] + '_sq'
        print('new_cols_2=', new_cols_2)
        
#         求二次方
        df_top10[new_cols_2] = df_top10[top_cols[0][i]] ** 2
#     计算三次方
        df_top10[new_cols_3] = df_top10[top_cols[0][i]] ** 3
#     计算平方根
        df_top10[new_cols_sq] = np.sqrt(df_top10[top_cols[0][i]]) 
        
    return df_top10

# 训练集和测试集都要添加 
train_part1_top10 = Polynomials_top10(train_part1, top_cols, top_num)
test_df_top10 = Polynomials_top10(test_df, top_cols, top_num)

new_cols_2= pay_price_s2
new_cols_2= stone_reduce_value_s2
new_cols_2= ivory_add_value_s2
new_cols_2= ivory_reduce_value_s2
new_cols_2= general_acceleration_reduce_value_s2
new_cols_2= stone_add_value_s2
new_cols_2= wood_add_value_s2
new_cols_2= general_acceleration_add_value_s2
new_cols_2= wood_reduce_value_s2
new_cols_2= training_acceleration_reduce_value_s2
new_cols_2= pay_price_s2
new_cols_2= stone_reduce_value_s2
new_cols_2= ivory_add_value_s2
new_cols_2= ivory_reduce_value_s2
new_cols_2= general_acceleration_reduce_value_s2
new_cols_2= stone_add_value_s2
new_cols_2= wood_add_value_s2
new_cols_2= general_acceleration_add_value_s2
new_cols_2= wood_reduce_value_s2
new_cols_2= training_acceleration_reduce_value_s2


In [95]:
# train_part1.head()
# train_part1_top10.head()
# test_df.head()
test_df_top10.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,wood_add_value_sq,general_acceleration_add_value_s2,general_acceleration_add_value_s3,general_acceleration_add_value_sq,wood_reduce_value_s2,wood_reduce_value_s3,wood_reduce_value_sq,training_acceleration_reduce_value_s2,training_acceleration_reduce_value_s3,training_acceleration_reduce_value_sq
0,14933,94,166415.0,138362.0,0,0,0,0,258522,90142,...,407.939947,9,27,1.732051,1.914404e+10,2.648808e+15,371.970429,0,0,0.0
1,14934,94,10000.0,600.0,0,0,0,0,10000,400,...,100.000000,0,0,0.000000,3.600000e+05,2.160000e+08,24.494897,0,0,0.0
2,14935,94,10000.0,3700.0,0,0,0,0,10000,2000,...,100.000000,0,0,0.000000,1.369000e+07,5.065300e+10,60.827625,0,0,0.0
3,14936,94,210000.0,0.0,200000,0,200000,0,610000,0,...,458.257569,435600,287496000,25.690465,0.000000e+00,0.000000e+00,0.000000,0,0,0.0
4,14937,94,11500.0,3700.0,0,0,0,0,11000,2000,...,107.238053,0,0,0.000000,1.369000e+07,5.065300e+10,60.827625,0,0,0.0


In [88]:
# train=0.1-fe.csv 添加重要特征（top 10）的多项式
train_part1_top10.to_csv(path_new + 'train=0.1-fe-top10.csv', index=False)
train_part1_top10.shape

(228800, 139)

In [89]:
# tap_fun_test-fe.csv 添加重要特征（top 10）的多项式
test_df_top10.to_csv(path_new + 'tap_fun_test-fe-top10.csv', index=False)
test_df_top10.shape

(828934, 138)

## 添加重要特征（top 20）的多项式

In [96]:
# 与目标值相关性最高的 20 个特征+自身 = 21 个
top_num = 21
top20_cols = get_top_col(sort2_coor, 'prediction_pay_price', top_num)
top20_cols
# 训练集和测试集都要添加 
train_part1_top20 = Polynomials_top10(train_part1, top20_cols, top_num)
test_df_top20 = Polynomials_top10(test_df, top20_cols, top_num)

threshold= 0.608037549401
top_cols= [Index(['prediction_pay_price', 'pay_price', 'stone_reduce_value',
       'ivory_add_value', 'ivory_reduce_value',
       'general_acceleration_reduce_value', 'stone_add_value',
       'wood_add_value', 'general_acceleration_add_value', 'wood_reduce_value',
       'training_acceleration_reduce_value', 'meat_reduce_value',
       'reaserch_acceleration_reduce_value', 'training_acceleration_add_value',
       'sr_infantry_hp_level', 'reaserch_acceleration_add_value',
       'meat_add_value', 'sr_cavalry_def_level', 'sr_shaman_hp_level',
       'sr_infantry_def_level', 'sr_shaman_def_level'],
      dtype='object')]
new_cols_2= pay_price_s2
new_cols_2= stone_reduce_value_s2
new_cols_2= ivory_add_value_s2
new_cols_2= ivory_reduce_value_s2
new_cols_2= general_acceleration_reduce_value_s2
new_cols_2= stone_add_value_s2
new_cols_2= wood_add_value_s2
new_cols_2= general_acceleration_add_value_s2
new_cols_2= wood_reduce_value_s2
new_cols_2= training_accelerati

In [97]:
# train_part1.head()
# train_part1_top20.head()
# test_df.head()
test_df_top20.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_cavalry_def_level_sq,sr_shaman_hp_level_s2,sr_shaman_hp_level_s3,sr_shaman_hp_level_sq,sr_infantry_def_level_s2,sr_infantry_def_level_s3,sr_infantry_def_level_sq,sr_shaman_def_level_s2,sr_shaman_def_level_s3,sr_shaman_def_level_sq
0,14933,94,166415.0,138362.0,0,0,0,0,258522,90142,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
1,14934,94,10000.0,600.0,0,0,0,0,10000,400,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
2,14935,94,10000.0,3700.0,0,0,0,0,10000,2000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
3,14936,94,210000.0,0.0,200000,0,200000,0,610000,0,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0
4,14937,94,11500.0,3700.0,0,0,0,0,11000,2000,...,0.0,0,0,0.0,0,0,0.0,0,0,0.0


In [98]:
# train=0.1-fe.csv 添加重要特征（top 20）的多项式
train_part1_top20.to_csv(path_new + 'train=0.1-fe-top20.csv', index=False)
train_part1_top20.shape

(228800, 169)

In [99]:
# tap_fun_test-fe.csv 添加重要特征（top 20）的多项式
test_df_top20.to_csv(path_new + 'tap_fun_test-fe-top20.csv', index=False)
test_df_top20.shape

(828934, 168)

## 去掉不付费的记录，只保留付费记录

In [124]:
# 去掉不付费的记录，只保留付费记录
def get_only_pay(df, is_high=True):
#     if is_high: # 高相关性的不包含 pay_count
#         df_pay = df.loc[bool(df['pay_price'] > 0 or df['prediction_pay_price'] > 0)]
#     else:
#         df_pay = df.loc[bool(df['pay_price'] > 0 or df['pay_count'] > 0 or df['prediction_pay_price'] > 0)]
    df_pay = df.loc[df['pay_price'] > 0]
    return df_pay

train_high_coor_pay = get_only_pay(train_high_coor, True)
train_high_coor_pay.shape

(4201, 49)

In [128]:
train_part1_pay = get_only_pay(train_part1, True)
train_part1_pay.shape

(4201, 109)

In [129]:
# 查看筛选之前的数量 
train_high_coor.shape
# train_part1.shape

(228800, 49)

In [130]:
# train=0.1-fe-coor0.3.csv 去掉不付费的记录，只保留付费记录
train_high_coor_pay.to_csv(path_new + 'train=0.1-fe-coor0.3-pay.csv', index=False)
# train=0.1-fe.csv 去掉不付费的记录，只保留付费记录 
train_part1_pay.to_csv(path_new + 'train=0.1-fe-pay.csv', index=False)